# Benzinga-Nachrichten-Verarbeitung

## Setup up

### Cluster spin up

In [1]:
use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    cwd="/content/drive/MyDrive/NewsTrading/trading_bot"
    %cd /content/drive/MyDrive/NewsTrading/trading_bot
    %pip install -r requirements_clean.txt

In [2]:
%cd /gxfs_work/cau/sunms534/trading_bot/

/gxfs_work/cau/sunms534/trading_bot


/gxfs_work/cau/sunms534/.conda/envs/my_pytorch_env/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import re
import pandas as pd
from src.config import config 
from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.display import clear_output

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)

from src.utils.dataframes import parallelize_dataframe, block_apply_factory
from src.preprocessing.news_parser import infer_author, filter_body, body_formatter

pd.set_option(
    'display.max_colwidth', 2000
)

## Experimental

In [10]:
news_csv = pd.read_csv("data/news/nasdaq_exteral_data.csv")

/scratch/SlurmTMP/sunms534.10454313/ipykernel_1091601/2219788147.py:1: DtypeWarning: Columns (3,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  news_csv = pd.read_csv("data/news/nasdaq_exteral_data.csv")


In [33]:
news_csv.rename(columns={
    "Date": "time",
    "Stock_symbol": "stocks",
    "Author": "author",
    "Article_title": "title",
    "Article":"body"
}, inplace=True)

In [44]:
N_0 = news_csv.shape

In [35]:
news_csv = news_csv.loc[:, ['time', 'stocks', 'author', 'title', 'body']]

Many time stamps don't contain intra-day information. Hence we need to set the intra-day time of these entries to the end of the day and mark them as special rows.

In [45]:
news_csv.dropna(subset=['time', 'stocks', 'body'], inplace=True)

In [46]:
N_1 = news_csv.shape

In [48]:
print(f"Dropped {N_0[0] - N_1[0]} entries due to NaNs.")

Dropped 13057521 entries due to NaNs.


In [49]:
news_csv['time'] = pd.to_datetime(news_csv['time'])

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
# OOM after starting second loop... something isn't being properly being garbage collected... maybe the child processes of multiprocessing in parallelize_dataframe?
%%time
for year in range(2023, 2024):
    print(f"{year}")
    df = pd.read_parquet(config.data.benzinga.raw + f"/story_df_raw_{year}.parquet")
    df["html_body"] = parallelize_dataframe(df["html_body"], block_apply_factory(body_formatter), n_cores=os.cpu_count())
    df = df.rename(columns={"html_body":"body"})
    df.to_parquet(config.data.benzinga.raw_html_parsed + f"/story_df_html_parsed{year}.parquet")
    clear_output(wait=True)

## Author-Inferenz

In [ ]:
ddf = pd.read_parquet(config.data.benzinga.raw_html_parsed)

In [ ]:
ddf.memory_usage(deep=True).sum() / 1024**3

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
%%time
ddf["inferred_author"] = None
ddf["inferred_author"] = ddf.body.progress_apply(infer_author)

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

In [ ]:
auhtor_value_counts.sum().diff()

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf.inferred_author.value_counts()

In [ ]:
ddf.inferred_author.value_counts().sum()

In [ ]:
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# Legacy code... channels only present in benzinga news and only after 2017 aswell
# Contains 100k rows
# earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x)]
# value counts for authors of earnings reports (contrast to value counts of all news articles)
# earnings_ddf.author.value_counts().head(10)

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt. Bzw. generell ist das eine gute Datensäuberungs-Maßnahme. Einerseits verhindern wir,
dass später Aktienkurse einer falschen Aktie zugeordnet wird. Andererseits werden dadurch evtl. auch weniger seriöse Nachrichten herausgefiltert, die nicht
die Kontaktadresszeile des Unternehmens am Ende besitzen, in dem der vollständige Unternehmensname vorkommt.

In [ ]:
mapper = pd.read_parquet(config.data.shared.ticker_name_mapper)

In [ ]:
ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]
ddf["company_name"] = ddf.stocks.progress_map(lambda x: mapper.company_names.loc[x]).astype(str)
ddf["short_name"] = ddf.stocks.progress_map(lambda x: mapper.short_name.loc[x]).astype(str)
print(f"Es verbleiben {ddf.shape[0]} Nachrichten, für die wir den Ticker zu einem Firmennamen aufgelösen konnten.")

In [ ]:
ddf.company_name

### Duplikate Entfernen

In [ ]:
%%time
samples_before = ddf.shape[0]
ddf = ddf.drop_duplicates()
samples_after = ddf.shape[0]
print(f"{samples_before=}, {samples_after=}")

### Firmennamen-Nachrichtenkörper-Verifikation

In [ ]:
def verify_company_name_in_body(df):
    return df.apply(lambda x: bool(re.search(x["short_name"],
                                             x.title + x["body"].replace("( )*\n( )*", " "),
                                             re.IGNORECASE)),
                    axis=1)

In [ ]:
%%time
mask = parallelize_dataframe(ddf, verify_company_name_in_body, n_cores=os.cpu_count())

print(f"Around {len(ddf.stocks.unique())} stocks before filtering and {len(ddf[mask].stocks.unique())} after")

In [ ]:
# Filter out faulty news
ddf = ddf[mask]

Bis zu 5k Nachrichten pro Firma, z.B. AT&T, was in 13 Jahren ca. einer Nachricht pro Tag entspricht. Wir wollen nicht das eine Firma mit vielen Junk-Nachrichten das Modell dominiert. Wobei das Modell hoffentlich dann auch die Junk-Nachrichten als solche erkennt und ignoriert. Eher wichtig noch einen `staleness`-Faktor, also wie ähnlich die Nachricht zu Vorhergegangenen ist (i.e. Nachrichten desselben Tages oder derselben Woche).

Kategorisieren von Nachrichten (mit Text2Topic, wie Salbrechter?) und eliminieren von Business/Strategic etc.
Im Falle von Text2Topic, versuche Estimates des Unternehmens von Dritten zu unterscheiden.

Wichtig!!! Unterscheide zwischen LERN-Phase und PRODUKTIONS-Phase.
Wir können z.B. CLS-Token in der Produktions-Phase vergleichen, in der Lern-Phase aber noch nicht.

Text2Vec -> Business category evtl. entfernen-> Intrastock variance average

## Make reduced ticker name mapping 

In [ ]:
ticker_name_mapper = pd.read_parquet(config.data.shared.ticker_name_mapper)

ticker_name_mapper_reduced = ddf[["stocks", "company_name", "short_name"]].drop_duplicates(keep="first")

ticker_name_mapper_reduced.to_parquet(config.data.shared.ticker_name_mapper_reduced)

print(f"From {ticker_name_mapper.shape[0]} to {ticker_name_mapper_reduced.shape[0]} tickers (reduced)")

## Parsing News Bodies


In [ ]:
from src.utils.time import convert_timezone
ddf["time"] = ddf["time"].progress_map(lambda x: convert_timezone(pd.to_datetime(x)))

In [ ]:
sample = ddf#.iloc[:10000, :]

In [ ]:
ddf.columns

In [ ]:
def block_apply_factory(func, axis=None):
    global _f
    def _f(s):
        # s can be series or dataframe
        print(' ', end='', flush=True)
        ret = s.progress_apply(func, axis=axis) if axis else s.progress_apply(func)
        return ret
    return _f

In [ ]:
%%time
ddf["parsed_body"] = parallelize_dataframe(sample, block_apply_factory(filter_body, axis=1), n_cores=os.cpu_count())

In [ ]:
ddf.to_parquet(config.data.benzinga.cleaned)

In [ ]:
### Analysis

In [ ]:
ddf.columns

In [ ]:
ddf.company_name.tail(10)

In [ ]:
ddf[["body", "parsed_body"]].tail(10).iloc[0:1, :]

In [ ]:
ddf[["body", "parsed_body"]].tail(10).iloc[0,1]

## Filtern von Newstiteln

## Voranstellen von gefilterten Newstiteln an Nachrichtenkörper

# Analyse

## Durschnittlichen Tokenlänge

### Timedeltas zwischen Nachrichtenmeldungen

In [ ]:
import plotly.express as px


In [ ]:
ddf = pd.read_parquet(config.data.benzinga.cleaned)

In [ ]:
ddf = ddf.sort_values("time")
tmp = ddf[["time", "stocks"]]
#### Adding timedeltas to the data frame
news_timedeltas = tmp.groupby("stocks").transform(lambda x: x.diff())
ddf.loc[:, "timedelta"] = news_timedeltas.time.fillna(pd.Timedelta(days=100))

In [ ]:
news_timedeltas = ddf.timedelta
news_timedeltas.iloc[0].components
timedelta_in_minutes = news_timedeltas.apply(lambda x: x.total_seconds() / 60)
px.histogram(timedelta_in_minutes)